In [1]:
import random
def get_n_states_at_random(n,numStates):
    states_list = [] 
    for i in range(n):
        rand_state = random.randint(0,numStates)
        while rand_state in states_list: 
            rand_state = random.randint(0,numStates)
        states_list.append(rand_state)
    return states_list 

In [2]:
def get_labels(prism_model_lines):
    labels={}
    text_to_find = "label"
    text_to_scrap = "WayPoint"
    text_to_scrap2 = "waypoint"
    for i in range(len(prism_model_lines)):
        line = prism_model_lines[i]
        if text_to_find in line:
            if text_to_scrap in line and text_to_scrap2 in line:
                wp_name = line[line.find("WayPoint"):line.rfind('"')]
                #print wp_name 
                wp_num = line[line.find("(")+1:line.find(")")]
                #print wp_num 
                wp_name = wp_name[len(text_to_scrap):]
                wp_num = wp_num[len(text_to_scrap2+"="):]
                #print wp_name 
                #print wp_num
                labels[int(wp_name)]=int(wp_num)
    return labels

In [3]:
def get_module_lines(prism_model_lines):
    start = -1
    end = -1
    startword = "module"
    endword = "endmodule"
    for i in range(len(prism_model_lines)):
        line = prism_model_lines[i]
        if endword in line: 
            end = i
        elif startword in line:
            start = i 
        if (start != -1 and end != -1):
            break 
    return (start+1,end,prism_model_lines[start+1:end])

In [4]:
def add_failure_to_states(state, module_lines):
    print "Adding failstates to "+str(state)
    state_line = "1.0:(waypoint'="+str(state)+")"
    rep_line = "p:(waypoint'="+str(state)+") + (1-p):(waypoint'=failstate)"
    new_module_lines = []
    for line in module_lines:
        if state_line in line:
            line = line.replace(state_line,rep_line)
        new_module_lines.append(line)
    return new_module_lines

In [5]:
def create_new_prism_model_lines(new_module_lines,module_start,module_end,prism_model_lines):
    new_prism_model = list(prism_model_lines)
    new_prism_model[module_start:module_end]=new_module_lines
    return new_prism_model

In [6]:
def write_model_to_file(filename,model_lines):
    f=open(filename,"w")
    f.writelines(model_lines)
    f.close()

def create_model_name(filename,suffix,model_ext):
    nfilename = filename+"_"+suffix+model_ext#filename+"_fs"+str(num_fail_states)+model_ext   
    return nfilename

In [7]:
def get_init_state_for_model_num(init_states,num_model):
    #init_states = [21,24,6,15,5,1,19,8]
    return init_states[num_model]

In [8]:
def modify_init_line(module_lines,num_model,init_states):
    init_line_text="waypoint"
    init_line_text2="init"
    for i in range(len(module_lines)):
        if (init_line_text in module_lines[i] and init_line_text2 in module_lines[i]):
            init_start_index = module_lines[i].find("init ")
            temptext = module_lines[i][:init_start_index+len("init ")]
            module_lines[i]=temptext+str(get_init_state_for_model_num(init_states,num_model))+";\n"
            break
    return module_lines        

In [9]:
def create_new_prism_model(folder,new_folder,filename,new_filename_suffix,prism_model_ext,numStates,
                           states_to_fail_at,num_models,init_states,convert_states_from_labels):
    #read in the model and extract the module bit
    prism_model_file = open(folder+filename+prism_model_ext)
    prism_model_lines = prism_model_file.readlines()
    prism_model_file.close()
    module_start,module_end,module_lines = get_module_lines(prism_model_lines)
    labels = get_labels(prism_model_lines)
    #generate states to fail at
    #states_to_fail_at=get_n_states_at_random(num_rand_states,numStates)
    if convert_states_from_labels:
        for i in range(len(states_to_fail_at)):
            states_to_fail_at[i]=labels[states_to_fail_at[i]]
        for i in range(len(init_states)):
            init_states[i]=labels[init_states[i]]
        print "New States To Fail At"
        print states_to_fail_at
        print "New Init States"
        print init_states
            
    #create the new model 
    for i in range(len(states_to_fail_at)):
        state_to_look_at = states_to_fail_at[i]
        new_module_lines=add_failure_to_states(state_to_look_at,module_lines)
        module_lines = new_module_lines
        
    new_prism_model_lines = create_new_prism_model_lines(new_module_lines,module_start,module_end,prism_model_lines)
    
    for i in range(num_models):
        new_prism_model_lines = modify_init_line(new_prism_model_lines,i,init_states)
        #write to file
        new_filename = create_model_name(new_folder+filename,new_filename_suffix+str(i),prism_model_ext)
        write_model_to_file(new_filename,new_prism_model_lines)

In [11]:
folder = "../decomp_tests/"
filename = "grid_3_topomap"#"grid_2_topomap"#"grid_map_topmap"
prism_model_ext = ".prism"
#we know that there are 0 to 28 states 
numStates = 33 
num_models = 4
init_states=[19,3,49,30]#[57,50,34,43]
states_to_fail_at=[8,22,27,28,36,47,59]#[63,59,64,61,62,60]
filename_suffix = "sim"

In [12]:
create_new_prism_model(folder,folder,filename,filename_suffix
                       ,prism_model_ext,numStates,states_to_fail_at,num_models,init_states,True)

New States To Fail At
[12, 27, 32, 33, 37, 55, 5]
New Init States
[25, 10, 53, 38]
Adding failstates to 12
Adding failstates to 27
Adding failstates to 32
Adding failstates to 33
Adding failstates to 37
Adding failstates to 55
Adding failstates to 5
